# Tratamento de dados GLM para a criação de uma composição audiovisual  

### Para que serve?

O códifo abaixo manipula para dados do satélite do tipo GLM (geostationary lightning mapper), que é um mapeador de relâmpagos em órbita geoestacionária. Ele possuiu o formato NetCDF e é feita uma transformação dos dados, colocando-os em um csv, com as informações de latitude, longitude e data/horário do acontecimento, essa planilha passa por um tratamento para podermos usá-la em uma composição audiovisual do tipo p5js, que utiliza dados em tempo real em até 24 horas da execução da requisição.

### Bibliotecas necessárias:

In [ ]:
from netCDF4 import Dataset                           # Read / Write NetCDF4 files
from datetime import timedelta, datetime, date        # Basic Dates and time types
import os                                             # Miscellaneous operating system interfaces
from osgeo import gdal                                # Python bindings for GDAL
import boto3                                          # Amazon Web Services (AWS) SDK for Python
from botocore import UNSIGNED                         # boto3 config
from botocore.config import Config                    # boto3 config
import pandas as pd
import pickle
import pytz                                           # fuso horário
import csv
import random
gdal.PushErrorHandler('CPLQuietErrorHandler')         # Ignore GDAL warnings

Foram acrescentadas as bibliotecas pickle, random e csv para podermos realizar o tratamento.

### Explicando a lógica

Primeiramente é deletado todos os arquivos já existentes nas pastas Samples (com as imagens e arquivos por mapeamento de área) e GLM (que contém as tabelas com todos os arquivos), já que, por execução são criados em média 25 arquivos na primeira pasta e longas tabelas na pasta GLM.

In [ ]:
def deletar_arquivos_pasta(pasta):
    # Percorre todos os arquivos da pasta
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo = os.path.join(pasta, nome_arquivo)

        # Verifica se é um arquivo
        if os.path.isfile(caminho_arquivo):
            # Deleta o arquivo
            os.remove(caminho_arquivo)
            print(f"Arquivo deletado: {caminho_arquivo}")

    print(f"Todos os arquivos da pasta {pasta} foram deletados.")

deletar_arquivos_pasta('GLM')
deletar_arquivos_pasta('Samples')

Após isso é declarada a função de downloaad dos dados NetCDF.

In [ ]:
def download_GLM(yyyymmddhhmnss, path_dest, bucket_name):

  os.makedirs(path_dest, exist_ok=True)

  year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y')
  day_of_year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%j')
  hour = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%H')
  min = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%M')
  seg = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%S')

  # Initializes the S3 client
  s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

  # File structure
  product_name = "GLM-L2-LCFA"
  prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}_G16_s{year}{day_of_year}{hour}{min}{seg}'

  # Seach for the file on the server
  s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

  # Check if there are files available
  if 'Contents' not in s3_result:
    print(f'No files found for the date: {yyyymmddhhmnss}, Product-{product_name}')
    return -1
  else:
    for obj in s3_result['Contents']:
      key = obj['Key']
      file_name = key.split('/')[-1].split('.')[0]
      if os.path.exists(f'{path_dest}/{file_name}.nc'):
        print(f'File {path_dest}/{file_name}.nc exists')
      else:
        print(f'Downloading file {path_dest}/{file_name}.nc')
        s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')
  return f'{file_name}'

Para utilizar os dados em um intervalo de tempo de 24 horas até o momento que o código é executado, utilizamos:

In [ ]:
# Exemple:
lat = -10
lon = -50

# Desired data:
input = "Samples"; os.makedirs(input, exist_ok=True)
output = "GLM"; os.makedirs(output, exist_ok=True)


#Dados no tempo real
fuso = pytz.timezone('America/Sao_Paulo')
now = datetime.now(fuso)

final_day = now.day
inicial_day = now - timedelta(days=1)
month = now.month
year = now.year
hours = now.hour
minutes = now.minute
seconds = now.second
bucket_name = 'noaa-goes16'

date_ini = str(datetime(year, month, inicial_day.day, hours, minutes))
date_end = str(datetime(year, month, final_day, hours, minutes))
primeiro = True


Uma mudança essencial foi a de alterar os intervalos de tempo em que os dados eram captados, o GOES-R capta informações a cada 20 segundos, o que leva a uma execução com muitos dados, chegando a captar mais de 400 mil informações. Assim foi alterado o timedelta no final do loop para ele captar informações a cada 1 hora.

In [ ]:
while (date_ini <= date_end):
    # Get the GLM Data
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')
    fileGLM = download_GLM(yyyymmddhhmnss, input, bucket_name)
    glm = Dataset(f'{input}/{fileGLM}.nc')

    f_lats = glm.variables['flash_lat'][:]
    f_lons = glm.variables['flash_lon'][:]

    if (primeiro):
        df_anterior = pd.DataFrame({"lat": f_lats, "lon": f_lons, "time": datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y-%m-%d %H:%M:%S')})
        primeiro = False

    else:
      df_1 = pd.DataFrame({"lat": f_lats, "lon": f_lons, "time": datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y-%m-%d %H:%M:%S')})
      df = pd.concat([df_anterior, df_1], ignore_index=True)
      df_anterior = df

    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(hours=1))

df.to_csv(f'{output}/flashs.csv')

df = pd.read_csv(f'{output}/flashs.csv')

# Looking for a lightning in the user's region
for i in range(len(df['lat'])):
    if (df['lat'][i] <= (lat + 0.5) and df['lat'][i] >= (lat - 0.5)) and (df['lon'][i] <= (lon + 0.5) and df['lon'][i] >= (lon - 0.5)):
        print("found lightning")



Ainda sim, a contagem final de dados fica por volta de 10 mil dados, o que ainda é muito para o cenário estudado. Então foi criada uma função para captar aleatoriamente 150 registros dos captados, esse parametro é usado como exemplo e pode ser alterado.

In [ ]:
def filtrar_dados(arquivo_csv, num_registros, arquivo_saida):
    registros_selecionados = []
    with open(arquivo_csv, 'r') as arquivo:
        leitor_csv = csv.reader(arquivo)
        cabeçalho = next(leitor_csv)  # Lê o cabeçalho do arquivo CSV

        # Lê todos os registros do arquivo e armazena-os em uma lista
        todos_registros = list(leitor_csv)

        # Verifica se o número de registros solicitado é maior do que o número total de registros
        if num_registros > len(todos_registros):
            num_registros = len(todos_registros)

        # Seleciona aleatoriamente os registros
        registros_selecionados = random.sample(todos_registros, num_registros)

    # Remove a primeira coluna dos registros selecionados para retirar os indices que não estão sequenciais devido
    # a escolha randomica
    registros_selecionados = [registro[1:] for registro in registros_selecionados]

    # Adiciona o cabeçalho aos registros selecionados
    registros_selecionados.insert(0, cabeçalho[1:])

    # Obtém o diretório do arquivo de saída
    diretorio_saida = os.path.dirname(arquivo_saida)

    # Cria o diretório de saída se ele não existir
    if not os.path.exists(diretorio_saida):
        os.makedirs(diretorio_saida)

    # Salva os registros selecionados em um novo arquivo CSV
    with open(arquivo_saida, 'w', newline='') as arquivo_saida_csv:
        escritor_csv = csv.writer(arquivo_saida_csv)
        escritor_csv.writerows(registros_selecionados)


    print(f"Registros selecionados salvos com sucesso no arquivo: {arquivo_saida}")


Outra necessidade encontrada foi a de achar os valores mínimos e máximos da primeira e segunda coluna (latitude e longitude) para poder usar esses como padrões para a normalização da visualização de dados da composição audiovisual. Para isso foi criado a função que capta esses metadados.

In [ ]:
def metadados(arquivo_csv):
    # Ler o arquivo CSV e extrair os valores das colunas
    primeira_coluna = []
    segunda_coluna = []
    with open(arquivo_csv, 'r') as arquivo:
        leitor_csv = csv.reader(arquivo)
        # Ignorar o cabeçalho
        next(leitor_csv)
        # Ler as colunas
        for linha in leitor_csv:
            primeira_coluna.append(float(linha[0]))
            segunda_coluna.append(float(linha[1]))

    # Criar um dicionário com os metadados
    metadados = {
        'maior_valor_primeira_coluna': max(primeira_coluna),
        'menor_valor_primeira_coluna': min(primeira_coluna),
        'maior_valor_segunda_coluna': max(segunda_coluna),
        'menor_valor_segunda_coluna': min(segunda_coluna)
    }


    print("Metadados dos limites de latitude e longitude encontrados.")

    #Printa os metadados
    for valor in metadados.values():
      print(valor)

Passagem de parametros e chamada da função:


In [ ]:
arquivo_csv = 'GLM/flashs.csv'
num_registros_selecionados = 150
arquivo_saida = 'GLM/flashs_filtro.csv'

filtrar_dados(arquivo_csv, num_registros_selecionados, arquivo_saida)
metadados(arquivo_saida)

## O que pode ser aprimorado?

Por enquanto, é necessário captar a longitude e latitude exata do usuário para acrescentarmos nesse script. É interessante também formular uma lógica em cima de qual informações são mais importantes para o contexto, já que muitos dados são deixados de lado visando um meio artístico e menos técnico.

O código abaixo foi retirado e manipulado a partir de scripts que podem ser encontrados em: https://github.com/isabellarigue/GaiaSenses/blob/main/docs/glm_csv.ipynb e https://geonetcast.wordpress.com/2021/02/25/vlab-processamento-de-dados-de-satelites-geoestacionarios-pre-curso/